In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacityworkspace
Azure region: eastus
Subscription id: bfdbbde2-f093-45f7-9b9d-c1721eed4782
Resource group: udacity


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "My-Cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os


In [4]:

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.1, 5.0),           # regularization
        "--max_iter": choice(20, 50, 80)    # max number of iterations used
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4,
                                     )

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_39a5031e-fab6-49f8-8e97-3e4e2cc9b74b
Web View: https://ml.azure.com/runs/HD_39a5031e-fab6-49f8-8e97-3e4e2cc9b74b?wsid=/subscriptions/bfdbbde2-f093-45f7-9b9d-c1721eed4782/resourcegroups/udacity/workspaces/udacityworkspace&tid=7dba2f36-129a-4d4a-9ab4-1e18c9f46061

Streaming azureml-logs/hyperdrive.txt

[2023-10-06T09:27:06.482231][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-10-06T09:27:07.0951879Z][SCHEDULER][INFO]Scheduling job, id='HD_39a5031e-fab6-49f8-8e97-3e4e2cc9b74b_0' 
[2023-10-06T09:27:07.1776974Z][SCHEDULER][INFO]Scheduling job, id='HD_39a5031e-fab6-49f8-8e97-3e4e2cc9b74b_1' 
[2023-10-06T09:27:07.3545615Z][SCHEDULER][INFO]Scheduling job, id='HD_39a5031e-fab6-49f8-8e97-3e4e2cc9b74b_2' 
[2023-10-06T09:27:07.281022][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-10-06T09:27:07.4693596Z][SCHEDULER][INFO]Scheduling job, id='HD_39a5031e-fab6-49f8-8e97-3e4e2cc9b74b_3' 
[2023-10-06T09

{'runId': 'HD_39a5031e-fab6-49f8-8e97-3e4e2cc9b74b',
 'target': 'My-Cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-10-06T09:27:05.701023Z',
 'endTimeUtc': '2023-10-06T09:50:42.913628Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '6684baa6-8ec2-4951-9bb9-0324baddf3e5',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'score': '0.9099295945617868',
  'best_child_run_id': 'HD_39a5031e-fab6-49f8-8e97-3e4e2cc9b74b_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_39a5031e-fab6-49f8-8e97-3e4e2cc9b74b_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValue

In [6]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n parameter values:',parameter_values)

print(best_run.get_file_names())
filename = 'best_hyper_drive_model'
best_model = best_run.register_model(model_name=filename, model_path="./")

Best Run Id:  HD_39a5031e-fab6-49f8-8e97-3e4e2cc9b74b_0

 Accuracy: 0.9099295945617868

 parameter values: ['--C', '4.087769151752101', '--max_iter', '80']
['azureml-logs/20_image_build_log.txt', 'logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2023-10-06-09', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

df = TabularDatasetFactory.from_delimited_files(path)

In [10]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(df)
# the output of the clean_data function seems to be aimed at the deprecated way of inputting training_data
# so I have to put it back together
df = pd.concat([x, y], axis=1)
datastore=ws.get_default_datastore()
training_data=TabularDatasetFactory.register_pandas_dataframe(df, datastore, 'training_data') 

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/4d0f2362-1a00-494b-9c10-ac6bb48e2862/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name="y",
    n_cross_validations=4,
    compute_target=cpu_cluster)

In [15]:
# Submit your automl run

automl_run = exp.submit(config=automl_config)
RunDetails(automl_run).show()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_e705b4e1-b18c-4a1f-abe0-d711542656f2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [16]:
# Retrieve and save your best automl model.

automl_run.wait_for_completion(show_output=False)

{'runId': 'AutoML_e705b4e1-b18c-4a1f-abe0-d711542656f2',
 'target': 'My-Cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-10-06T09:56:28.968575Z',
 'endTimeUtc': '2023-10-06T10:32:35.537041Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'My-Cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"bfdbbde2-f093-45f7-9b9d-c1721eed4782","resource_group":"udacity","workspace_name":"udacityworkspace","region":"eastus","compute_target":"My-Cluster","spark_service":null,"azure_servi

In [17]:
best_run, best_model = automl_run.get_output()
best_run.register_model(model_name= 'best_model_automl.pkl', model_path= './outputs/')

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Model(workspace=Workspace.create(name='udacityworkspace', subscription_id='bfdbbde2-f093-45f7-9b9d-c1721eed4782', resource_group='udacity'), name=best_model_automl.pkl, id=best_model_automl.pkl:1, version=1, tags={}, properties={})

In [18]:
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])

Best Run Id:  AutoML_e705b4e1-b18c-4a1f-abe0-d711542656f2_30

 Accuracy: 0.9174809634038089


In [19]:
print('\n all metrics best run:', best_run_metrics)


 all metrics best run: {'precision_score_weighted': 0.9129879486576112, 'precision_score_macro': 0.799744697961511, 'recall_score_weighted': 0.9174809634038089, 'recall_score_micro': 0.9174809634038089, 'balanced_accuracy': 0.7621620218935404, 'average_precision_score_weighted': 0.9557099604930589, 'AUC_weighted': 0.9482484630791549, 'norm_macro_recall': 0.524324043787081, 'f1_score_micro': 0.9174809634038089, 'AUC_micro': 0.9809741268604837, 'f1_score_weighted': 0.9147389870474606, 'f1_score_macro': 0.7790121179842777, 'average_precision_score_micro': 0.9817399959144001, 'matthews_correlation': 0.5605221035891439, 'weighted_accuracy': 0.9560607643648366, 'precision_score_micro': 0.9174809634038089, 'accuracy': 0.9174809634038089, 'log_loss': 0.3074948026013571, 'recall_score_macro': 0.7621620218935404, 'AUC_macro': 0.948248463079155, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_e705b4e1-b18c-4a1f-abe0-d711542656f2_30/accuracy_table', 'average_precision_score_macro': 

In [20]:
# Proof of cluster cleanup
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, deleting it.")
    cpu_cluster.delete()
except ComputeTargetException:
    print("Cluster not found.")

Found existing cluster, deleting it.
